### Import packages

In [63]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm
import fitz  # PyMuPDF
from pathlib import Path

In [64]:
import os
print(os.getcwd())

/workspaces/llm-zoomcamp/mai/notebooks


## Load documents in good format

In [65]:
with open('../context/life_with_hope/structured/steps.json', 'r') as f: 
    steps = json.load(f)

## Create Index Settings for Elasticsearch

### Note: Looks like persists so trips an error if recreating in same instance?

In [66]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "step":        {"type": "integer"},
            "title":       {"type": "text"},
            "text":        {"type": "text"},
            "source":      {"type": "keyword"},
            "page_start":  {"type": "integer"},
            "page_end":    {"type": "integer"},
            "tags":        {"type": "keyword"}
        }
    }
}

index_name = "life-with-hope-steps"

# Create the index (will error if it already exists)
es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'life-with-hope-steps'})

In [76]:
# innitialize the conversation history 
current_conversation_history = []

In [32]:
# get spiritual explorer persona
def get_spiritual_explorer_persona():
    """Get the spiritual_explorer persona specifically"""
    with open('../context/personas/personas.json', 'r') as f:
        personas = json.load(f)
    return next(p for p in personas if p['persona_id'] == 'spiritual_explorer')

In [37]:
# RAG and Prompt Functions
def build_initial_conversation_prompt(query, conversation_history=None):
    """Build prompt for initial conversation using spiritual_explorer persona from JSON"""
    
    # Get spiritual_explorer persona from file
    persona = get_spiritual_explorer_persona()
    
    # Build conversation context if history exists
    history_text = ""
    if conversation_history:
        recent_history = conversation_history[-4:]  # Last 2 exchanges
        for msg in recent_history:
            role = "User" if msg["role"] == "user" else "Sponsor"
            history_text += f"{role}: {msg['content']}\n"
        history_text = f"\nRecent conversation:\n{history_text}"
    
    prompt_template = f"""You are an experienced MA sponsor meeting someone for their very first time at Marijuana Anonymous.

Your communication style: {persona['language_style']} 
Your approach: {persona['description']}

This is an initial meeting conversation. Your goals:
1. Welcome them warmly and create a safe space
2. Learn their name
3. Understand what brought them to MA today  
4. Gently assess their relationship with marijuana
5. Begin to understand their spiritual openness (without being pushy)
6. Determine if they're ready to start recovery work

Keep responses conversational and warm (2-3 sentences max). Don't rush through questions - let the conversation flow naturally.{history_text}

User says: "{query}"

Respond as their sponsor:"""
    
    return prompt_template

In [9]:
def llm_initial(prompt, model="gpt-4o"):
    """LLM call for initial conversations"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a warm, experienced MA sponsor. Keep responses natural and conversational."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [ ]:
def rag_initial_conversation(query, conversation_history=None):
    """RAG function for initial conversations"""
    prompt = build_initial_conversation_prompt(query, conversation_history)
    answer = llm_initial(prompt)
    # Update conversation history
    current_conversation_history.append({"sender": "user", "content": query})
    current_conversation_history.append({"sender": "ai_sponsor", "content": answer})
    
    return answer

In [100]:
rag_initial_conversation("I used to be a believe. now i dont know. but anyways i gotta go man. see you around",conversation_history=current_conversation_history)


"I completely understand. It's all about taking things one step at a time, and I'm here whenever you're ready to explore further. Take care, and I'll be here when you're ready for another chat. See you around!"

In [131]:
current_conversation_history

[]